In [1]:
import numpy as np 
import sys
import os
import matplotlib.pyplot as plt 
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/openmd")
from SimulatorLJ import SimulatorLJ as Simulator


In [15]:

def force_lj_fast( positions, constants, box_length):
    """
    :param positions:
    :type positions:
    :param constants:
    :type constants:
    :param box_length:
    :type box_length:
    :return:
    :rtype:
    """
    
    epsilon, sigma = constants
    num = positions.shape[0]
    force = np.zeros((num, num, 3))
    for i in range(0, num - 1):
        for j in range(i + 1, num):
            ###############
            difference = positions[j, :] - positions[i, :]
            distance = np.linalg.norm(difference)
            if distance > box_length / 2:
                distance -= box_length / 2
            elif distance <= -box_length / 2:
                distance += box_length / 2
            
            #################
            if distance == 0: 
                force[i,j] = 0 
                force[j,i] = 0
                break
            lj_part = (sigma / distance) ** 6
            lj_part_two = lj_part ** 2
            factor = 24 * epsilon
            factor_two = 2 * factor
            force[i, j, :] = (factor_two * lj_part_two - factor * lj_part) * (difference/distance)
            force[j, i, :] -= force[i, j, :]
    #print(np.sum(force, axis=1))
    return np.sum(force, axis=1)

In [16]:
dt = 0.001
positions = np.load("test_samples/test_sample__n_50_s_0.25_eps_0.25_.npy")
velocities = np.zeros(positions.shape)
constants = [1.0, 0.5]
box_length = 6
sim = Simulator(
    path = r"./output",
    title = "test_1",
    mass = 4,
    sim_time= 1000*dt ,
    time_step = dt,
    initial_values = [positions,velocities],
    box_length = box_length,
    force=None,
    force_constants=constants,
    integrator=None,   
    periodic=True,
)
sim.force = force_lj_fast
sim.force

<function __main__.force_lj_fast(positions, constants, box_length)>

In [151]:

def distance_new( positions, constants, box_length):
    """
    :param positions:
    :type positions:
    :param constants:
    :type constants:
    :param box_length:
    :type box_length:
    :return:
    :rtype:
    """
    
    epsilon, sigma = constants
    sigma_six = sigma**6
    prefactor_lj = 24*epsilon*sigma_six
    num = positions.shape[0]
    nvdim = positions.shape[1] # get nvdim for row major writing of matrices 
    position_rm = np.ravel(positions) #row major
    
    dmRM_CPU=np.zeros((num, num),dtype=np.float64,order='C')
    dmRM_CPU = np.ravel(dmRM_CPU)
    FRM_CPU = np.ravel(np.zeros((num, nvdim),dtype=np.float64,order='C'))
    
    
    for i in range(num):
        for j in range(i+1,num):
            dist = 0
            for k in range(nvdim):
                ab = position_rm[i*nvdim + k] - position_rm[j*nvdim + k]
                dist += ab*ab
            temp_dmRM = 1/np.sqrt(dist)
            temp_dmRM2 = temp_dmRM*temp_dmRM
            temp_dmRM6 = temp_dmRM2*temp_dmRM2*temp_dmRM2
            temp_dmRM8 = temp_dmRM6*temp_dmRM2
            for k in range(nvdim):
                ab = position_rm[i*nvdim + k] - position_rm[j*nvdim + k]
                #print(i*num + k)
                FRM_CPU[i*num + k] = (prefactor_lj/temp_dmRM8)*(((2*sigma_six)/temp_dmRM6)-1) * ab
            
    return FRM_CPU

In [152]:
positions = np.random.random((2,3))

In [153]:
%timeit -n 30 distances_new = distance_new( positions, constants, box_length)

35 µs ± 14.3 µs per loop (mean ± std. dev. of 7 runs, 30 loops each)


In [155]:
distances_new = distance_new( positions, constants, box_length)
print(diffdis)


[ 0.3944684   0.          0.         -0.01754702  0.          0.
  0.          0.          0.        ]


In [42]:
positions = np.random.random((3,3))

In [160]:
from numba import njit
@njit()
def distance_old( positions, constants, box_length):
    positions = positions.T
    epsilon, sigma = constants
    sigma_six = sigma**6
    prefactor_lj = 24*epsilon*sigma_six
    num = positions.shape[1]
    distances_sq = np.zeros((num, num))
    forces = np.zeros((positions.shape[0], num))
    for i in range(0, num - 1):
        for j in range(i + 1, num):
            ###############
            difference = positions[:, j] - positions[:, i]
            distance = np.sqrt(difference[0]**2 + difference[1]**2 + difference[2]**2)
            distance = 1/distance
            distance_sq = distance*distance
            distance_six = distance_sq*distance_sq*distance_sq
            distance_eight = distance_six*distance_sq
            prefactor = prefactor_lj*distance_eight
            bracket = ((2*sigma)*distance_six - 1)
            result = prefactor*bracket
            force = result * difference
            forces[:, i] += force
            forces[:, j] -= force 
            #distances_sq[i, j] = inv_dist**2
            
            #distances[j, i] = distance
            
            #if distance > box_length / 2:
            #    distance -= box_length / 2
            #elif distance <= -box_length / 2:
            #    distance += box_length / 2
            
            #################

    return forces

In [161]:
constants = [1,1]
positions = np.array([[0,0,0], [1,1, 2]])
distances_old = distance_old( positions, constants, box_length)
print(distances_old)

/home/julian/anaconda3/envs/openmd/lib/python3.10/site-packages/numba/core/ir_utils.py:2152: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'constants' of function 'distance_old'.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "../../../../../../tmp/ipykernel_5355/597338373.py", line 2:
<source missing, REPL/exec in use?>

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


[[-0.01834705  0.01834705]
 [-0.01834705  0.01834705]
 [-0.0366941   0.0366941 ]]


In [142]:
constants = [1,1]
positions = np.array([[0,0,0], [1,0, 0]])
distances_old = distance_old( positions, constants, box_length)
print(distances_old)

[[ 24. -24.]
 [  0.   0.]
 [  0.   0.]]


In [143]:
distance = np.sqrt(3)
prefactor = 24/distance**8
bracket = (2/distance**6 - 1)
result = prefactor*bracket
result 

-0.27434842249657077

In [144]:
constants = [1,1]
distances_old = distance_old( positions, constants, box_length)
print(distances_old)

[[ 24. -24.]
 [  0.   0.]
 [  0.   0.]]


In [162]:
positions = np.random.random((100,3))
%timeit -n 30 distances_old = distance_old( positions, constants, box_length)

/home/julian/anaconda3/envs/openmd/lib/python3.10/site-packages/numba/core/ir_utils.py:2152: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'constants' of function 'distance_old'.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "../../../../../../tmp/ipykernel_5355/597338373.py", line 2:
<source missing, REPL/exec in use?>

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


The slowest run took 17.42 times longer than the fastest. This could mean that an intermediate result is being cached.
3.79 ms ± 6.48 ms per loop (mean ± std. dev. of 7 runs, 30 loops each)


In [65]:
distances_new = distance_new( positions, constants, box_length)
distances_old = distance_old( positions, constants, box_length)
np.allclose(np.ravel(distances_old), distances_new) 

ValueError: operands could not be broadcast together with shapes (300,) (10000,) 

In [2]:
import pycuda.autoinit
import pycuda.driver as drv
from pycuda.compiler import SourceModule
from pycuda import gpuarray
from math import sqrt

In [3]:
MAX_THREADS_PER_BLOCK = drv.Device(0).get_attribute(pycuda._driver.device_attribute.MAX_THREADS_PER_BLOCK)
BLOCK_SIZE = int(sqrt(MAX_THREADS_PER_BLOCK))
print(MAX_THREADS_PER_BLOCK,BLOCK_SIZE)
BLOCK_SIZE_X = BLOCK_SIZE
BLOCK_SIZE_Y = BLOCK_SIZE #loops 

1024 32


In [12]:
#!/usr/bin/env python

import sys
import pycuda.autoinit
import pycuda.driver as drv
from pycuda.compiler import SourceModule
from pycuda import gpuarray
from math import sqrt
import numpy as np


kerneltemplate = SourceModule("""
#include <math.h>
__global__ void
gpuDM (float *dm, float *x, int n_dim, int n_vdim){

    extern __shared__ float a[];
    float dist;
    int tidx;

    int idx = threadIdx.x + blockDim.x*blockIdx.x;

    for (int r=0; r < n_dim; r++) {
        dist = 0;
        for (int i=0; i<=n_vdim/blockDim.x; i++){
            tidx = i*blockDim.x+threadIdx.x;
            if (tidx < n_vdim)
                a[tidx] = x[r*n_vdim + tidx];
        }
        __syncthreads();

        for(int i=0; i < n_vdim && idx < n_dim; i++){
            dist = a[i] - x[idx*n_vdim + i];
        }

        if ( idx < n_dim ){
            dm[idx*n_dim+r] = dist;
	}
        __syncthreads();
    }
}

__global__ void
gpuForces (float *fx, float *fy, float *fz, float *xrm, float *yrm, float *zrm, float epsilon, float sig, int n_dim){
    
    float prefactor, dist_seven, dist_thirteen;
    float sigma_six = pow(sig, 6);
    float prefactor_lj = 24*epsilon*sigma_six;
    

    int idx = threadIdx.x + blockDim.x*blockIdx.x;
    
    if ( (idx < n_dim) && ((xrm[idx]!=0) || (yrm[idx]!=0) || (zrm[idx]!=0)) ){
        float dist = 1/sqrtf(xrm[idx]*xrm[idx] + yrm[idx]*yrm[idx] + zrm[idx]*zrm[idx]);
        float distance_two = dist*dist;
        float distance_six = pow(distance_two,3);
        float distance_eight = distance_two*distance_six;
        float prefactor = prefactor_lj*distance_eight;
        float bracket = ((2*sig)*distance_six - 1);
        float result = prefactor*bracket;
        
        fx[idx] = result*xrm[idx];
        fy[idx] = result*yrm[idx];
        fz[idx] = result*zrm[idx];
        
    }
    __syncthreads();
}
__global__ void
gpuForces (float *fx, float *fy, float *fz, float *positions, float epsilon, float sig, int n_dim){
    float prefactor, distance, distance_sq, distance_six, distance_eight, prefactor, bracket, result;
    float sigma_six = pow(sig, 6);
    float prefactor_lj = 24*epsilon*sigma_six;
    
    for(int i =0; i<num; i++){
        for(int j = i+1; j<num; j++){
            ###############
            dx = positions[j*3] -   positions[i*3]
            dy = positions[j*3+1] - positions[i*3+1]
            dz = positions[j*3+2] - positions[i*3+2]
            distance = sqrtf(dx*dx + dy*dy + dz*dz)
            distance = 1/distance
            distance_sq = distance*distance
            distance_six = distance_sq*distance_sq*distance_sq
            distance_eight = distance_six*distance_sq
            prefactor = prefactor_lj*distance_eight
            bracket = ((2*sigma)*distance_six - 1)
            result = prefactor*bracket
            fx[i] += result * dx
            fy[i] += result * dy
            fz[i] += result * dz
            fx[j] -= result * dx
            fy[j] -= result * dy
            fz[j] -= result * dz
            
        }
    }
    
    
    
}
""")
dm_gpu = kerneltemplate.get_function("gpuDM")
force_gpu = kerneltemplate.get_function("gpuForces")
epsilon, sigma = np.int32(constants)
MAX_THREADS_PER_BLOCK = drv.Device(0).get_attribute(pycuda._driver.device_attribute.MAX_THREADS_PER_BLOCK)
BLOCK_SIZE = int(sqrt(MAX_THREADS_PER_BLOCK))
size_of_float = 4
#X = positions.astype(p.float64,order='C')
XRM = np.ravel(positions[:,0])
YRM = np.ravel(positions[:,1])
ZRM = np.ravel(positions[:,2])
ndim = np.shape(positions)[0]
nvdim = np.shape(positions)[1]


dm = np.zeros((ndim, ndim),dtype=np.float32,order='C')
dmRM = np.ravel(dm)

XRM_gpu = gpuarray.to_gpu(XRM)
YRM_gpu = gpuarray.to_gpu(YRM)
ZRM_gpu = gpuarray.to_gpu(ZRM)

dmXRM_gpu = gpuarray.empty(shape=dmRM.shape, dtype=np.float32)
dmYRM_gpu = gpuarray.empty(shape=dmRM.shape, dtype=np.float32)
dmZRM_gpu = gpuarray.empty(shape=dmRM.shape, dtype=np.float32)
smem_size = ndim*nvdim*size_of_float  # float memory 

dm_gpu(dmXRM_gpu, XRM_gpu, np.int32(ndim), np.int32(1), block=(32,1,1), grid=(1,1,1), shared=smem_size)
dm_gpu(dmYRM_gpu, YRM_gpu, np.int32(ndim), np.int32(1), block=(32,1,1), grid=(1,1,1), shared=smem_size)
dm_gpu(dmZRM_gpu, ZRM_gpu, np.int32(ndim), np.int32(1), block=(32,1,1), grid=(1,1,1), shared=smem_size)
# calculate forces
fx_gpu = gpuarray.empty(shape=dmRM.shape, dtype=np.float32)
fy_gpu = gpuarray.empty(shape=dmRM.shape, dtype=np.float32)
fz_gpu = gpuarray.empty(shape=dmRM.shape, dtype=np.float32)
force_gpu(fx_gpu, fy_gpu, fz_gpu, dmXRM_gpu, dmYRM_gpu, dmZRM_gpu, np.int32(1), np.int32(1), np.int32(ndim), block=(1,1,1), grid=(1,1,1))
fx_gpu.get() 
fy_gpu.get() 
fz_gpu.get() 
print(XRM_gpu.get())





[0 1]


/tmp/ipykernel_12688/1604596873.py:12: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu(36): warning #177-D: variable "prefactor" was declared but never referenced

kernel.cu(36): warning #177-D: variable "dist_seven" was declared but never referenced

kernel.cu(36): warning #177-D: variable "dist_thirteen" was declared but never referenced


  kerneltemplate = SourceModule("""


In [10]:
def lj_force_gpu(positions, constants, box_length, dm_gpu, force_gpu):
# get device information
    epsilon, sigma = np.int32(constants)
    MAX_THREADS_PER_BLOCK = drv.Device(0).get_attribute(pycuda._driver.device_attribute.MAX_THREADS_PER_BLOCK)
    BLOCK_SIZE = int(sqrt(MAX_THREADS_PER_BLOCK))
    size_of_float = 4
    #X = positions.astype(p.float64,order='C')
    XRM = np.ravel(positions[:,0])
    YRM = np.ravel(positions[:,1])
    ZRM = np.ravel(positions[:,2])
    ndim = np.shape(positions)[0]
    nvdim = np.shape(positions)[1]


    dm = np.zeros((ndim, ndim),dtype=np.float32,order='C')
    dmRM = np.ravel(dm)

    XRM_gpu = gpuarray.to_gpu(XRM)
    YRM_gpu = gpuarray.to_gpu(YRM)
    ZRM_gpu = gpuarray.to_gpu(ZRM)

    dmXRM_gpu = gpuarray.empty(shape=dmRM.shape, dtype=np.float32)
    dmYRM_gpu = gpuarray.empty(shape=dmRM.shape, dtype=np.float32)
    dmZRM_gpu = gpuarray.empty(shape=dmRM.shape, dtype=np.float32)
    smem_size = ndim*nvdim*size_of_float  # float memory 

    dm_gpu(dmXRM_gpu, XRM_gpu, np.int32(ndim), np.int32(1), block=(32,1,1), grid=(1,1,1), shared=smem_size)
    dm_gpu(dmYRM_gpu, YRM_gpu, np.int32(ndim), np.int32(1), block=(32,1,1), grid=(1,1,1), shared=smem_size)
    dm_gpu(dmZRM_gpu, ZRM_gpu, np.int32(ndim), np.int32(1), block=(32,1,1), grid=(1,1,1), shared=smem_size)
    fx_gpu = gpuarray.empty(shape=dmRM.shape, dtype=np.float32)
    fy_gpu = gpuarray.empty(shape=dmRM.shape, dtype=np.float32)
    fz_gpu = gpuarray.empty(shape=dmRM.shape, dtype=np.float32)
    force_gpu(fx_gpu, fy_gpu, fz_gpu, dmXRM_gpu, dmYRM_gpu, dmZRM_gpu, np.int32(1), np.int32(1), np.int32(ndim), block=(1,1,1), grid=(1,1,1))
    fx_gpu.get() 
    fy_gpu.get() 
    fz_gpu.get() 
    print(XRM_gpu.get())
    print(YRM_gpu.get())
    print(ZRM_gpu.get())

In [11]:
box_length = 3
constants = [1,1]
positions = np.array([[0,0,0], [1,1, 2]])
distances_old = lj_force_gpu( positions, constants, box_length, dm_gpu , force_gpu)
print(distances_old)

[0 1]
[0 1]
[0 2]
None


In [8]:
sim.dm_gpu = dm_gpu 

NameError: name 'sim' is not defined

In [10]:
for i in range(4): 
    x1 = i*3
    y1 = i*3 + 1
    z1 = i*3 + 2
    print("particle 1 is", i)
    print(x1,y1,z1)
    for j in range(i+1, 4): 
        x2 = j*3
        y2= j*3 + 1
        z2 = j*3 + 2
        print("particle 2is, ",j)
        print(x2,y2,z2)
        


particle 1 is 0
0 1 2
particle 2is,  1
3 4 5
particle 2is,  2
6 7 8
particle 2is,  3
9 10 11
particle 1 is 1
3 4 5
particle 2is,  2
6 7 8
particle 2is,  3
9 10 11
particle 1 is 2
6 7 8
particle 2is,  3
9 10 11
particle 1 is 3
9 10 11


In [65]:
from numba import njit
@njit()
def force_pre_cuda( positions, constants, box_length):
    num = positions.shape[0]
    positions = np.ravel(positions)
    epsilon= constants[0]
    sigma = constants[1]
    sigma_six = sigma**6
    prefactor_lj = 24*epsilon*sigma_six
    fx = np.zeros(num)
    fy = np.zeros(num)
    fz = np.zeros(num)
    for i in range(0, num - 1):
        for j in range(i + 1, num):
            ###############
            dx = positions[j*3] -   positions[i*3]
            dy = positions[j*3+1] - positions[i*3+1]
            dz = positions[j*3+2] - positions[i*3+2]
            distance = np.sqrt(dx*dx + dy*dy + dz*dz)
            distance = 1/distance
            distance_sq = distance*distance
            distance_six = distance_sq*distance_sq*distance_sq
            distance_eight = distance_six*distance_sq
            prefactor = prefactor_lj*distance_eight
            bracket = ((2.*sigma_six)*distance_six - 1.)
            result = prefactor*bracket
            fx[i] += result * dx
            fy[i] += result * dy
            fz[i] += result * dz
            fx[j] -= result * dx
            fy[j] -= result * dy
            fz[j] -= result * dz
    return fx, fy, fz

In [77]:
constants = np.array([1,1]).astype(np.int32, order="F")
positions = np.array([[0,0,0], [1,1, 2]]).astype(np.int32, order="F")
box_length = 3
distances_old = force_pre_cuda( positions, constants, box_length)
print(distances_old)

(array([-0.01834705,  0.01834705]), array([-0.01834705,  0.01834705]), array([-0.0366941,  0.0366941]))


In [74]:
positions = np.random.random((1000,3))
%timeit -n 100 distances_new = force_pre_cuda( positions, constants, box_length)



6.36 ms ± 435 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [75]:

dt = 0.000002
positions = np.load("test_samples/test_sample__n_128_s_0.25_eps_0.25_.npy")
velocities = np.zeros(positions.shape)
constants = np.array([0.1, 0.1])
box_length = np.int32(8)
sim = Simulator(
    path = r"./output",
    title = "test_1",
    mass = 1,
    sim_time= 10000*dt ,
    time_step = dt,
    initial_values = [positions,velocities],
    box_length = box_length,
    force=None,
    force_constants=constants,
    integrator=None,   
    periodic=True,
)
sim.force = force_pre_cuda
sim.force

CPUDispatcher(<function force_pre_cuda at 0x7ff860f0e440>)

In [76]:
positions, velocities = sim.simulate()

TypeError: unsupported operand type(s) for /: 'tuple' and 'int'